In [ ]:
%matplotlib inline

In [ ]:
from typing import Callable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import optimize

# PHYS 395 - week 4

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Root finding

Before looking at examples where root finding is useful, we're going to consider a number of root finding methods.

## Bisection method

Here we will use the bisection method to find a root of $f(x) = \sin(\cosh(2 x))$ on the interval $[0, 1]$.

Let's first look at $f$ on the interval.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot function
xs = np.linspace(0, 1, 500)

plt.plot(xs, np.sin(np.cosh(2 * xs)))
plt.grid(alpha=0.4)

ax.set_xlabel("x");

It looks like $f$ has one root on this interval at $x \approx 0.9$.

Let's code our own version of the bisection algorithm.

In [ ]:
def bisection_method(
    start: float, end: float, f: Callable[[float], float], tol: float = 1e-14
) -> float:
    """Find a root of an interval using the bisection method.
    
    Note that this can be made a bit more efficient, but there's
    no real need since there's already an existing SciPy method
    that implements this more efficiently.
    """
    # Evaluate endpoints
    f_start = f(start)
    f_end = f(end)

    # Check if either of the endpoints are zeros
    if f_start == 0:
        return start

    if f_end == 0:
        return end

    # If difference of endpoints is within tolerance,
    # return the start val
    if end - start < tol:
        return start

    # Get middle point and evaluate it
    middle = (start + end) / 2
    f_middle = f(middle)

    # Check if the middle is zero
    if f_middle == 0:
        return middle

    # Now bisect
    if f_start * f_middle < 0:
        return bisection_method(start, middle, f, tol)

    return bisection_method(middle, end, f, tol)

Let's try using the above function to find the root of $f$.

In [ ]:
x = bisection_method(0, 1, lambda x: np.sin(np.cosh(2 * x)))

print("x = %.10f" % x)

This is consistent with our guess. Now let's try finding the root using SciPy's version of the bisection algorithm.

In [ ]:
_, r = optimize.bisect(a=0, b=1, f=lambda x: np.sin(np.cosh(2 * x)), full_output=True)

print(r)

## Newton-Raphson method

For the Newton-Raphson (NR) method, instead of passing in a range in which to find a root, we pass in an initial guess. Going back to the plot of $f$ above, anything near $x = 0.5$ would be a bad initial guess, where the derivative of $f$ is either small or zero.

We also need to pass in the derivative to the NR method. Using the chain rule, we have

\begin{equation}
    \frac{df}{dx} = 2 \sinh (2 x) \cos( \cosh(2 x))
    .
\end{equation}

Let's use the NR method to find the root of $f$, giving it our initial guess of $x = 0.9$.

In [ ]:
_, r = optimize.newton(
    func=lambda x: np.sin(np.cosh(2 * x)),
    fprime=lambda x: 2 * np.sinh(2 * x) * np.cos(np.cosh(2 * x)),
    x0=0.9,
    full_output=True,
)

print(r)

As we can see by the number of iterations taken, NH is much faster than bisection! 

## Secant method

For the secant method, we essentially perform the NR method, but instead of explicitly using the derivative, we use a linear approximation of it. Let's see how well it performs.

In [ ]:
_, r = optimize.newton(func=lambda x: np.sin(np.cosh(2 * x)), x0=0.9, full_output=True,)

print(r)

For our function, it only required one more iteration, but also used one fewer function call.

## Brent's method

Now we'll try using Brent's method.

In [ ]:
_, r = optimize.brentq(a=0, b=1, f=lambda x: np.sin(np.cosh(2 * x)), full_output=True)

print(r)

This is much faster than the bisection method, but still slower than the NR or secant method. However, this has the advantage in that we don't have to provide an initial guess.

## General purpose root finding routine

Now we'll use SciPy's general purpose routine. We need to provide an initial guess for this, so we'll use the same one we used before.

In [ ]:
sol = optimize.root(fun=lambda x: np.sin(np.cosh(2 * x)), x0=0.9)

print(sol)

Unfortunately I don't know how to interpret the output of this function, so I can't write about how well it performed.

# Root finding applications

Now we'll look at several applications of root finding.

## Lagrange points

For the setup for this problem, see the lab script. Essentially what we need to do is find $r$ such that the following equation holds:

\begin{equation}
    \frac{G M}{r^2} - \frac{G m}{(R - r)^2} = \omega^2 r
    ,
\end{equation}

where $G$ is the gravitational constant; $M$ and $m$ are the masses of the Earth and Moon, respectively; $R$ is the  distance between the Earth and Moon; and $\omega$ is the angular frequency of the satellite.

In [ ]:
G = 6.674e-11  # m^3 / (kg s^2)
M = 5.974e24  # kg
m = 7.348e22  # kg
R = 3.844e8  # m
omega = 2.662e-6  # 1 / s

r = optimize.brentq(
    a=1, b=R - 1, f=lambda r: G * M / r ** 2 - G * m / (R - r) ** 2 - omega ** 2 * r
)

print("r = %e" % r)

In terms of $R$, we have

In [ ]:
print("r = %.2fR" % (r / R))

## Particle in a finite square well